In [4]:
import numpy as np
import torch
import pickle
import io
import warnings
warnings.filterwarnings('ignore')

MODEL_ARGS = (0.1, 5, 0.0, 2, 0.0, 5) # learning rate - epochs - lambda - batch size - weight decay - num folds
NUM_FOLDS = 5

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)
        
def get_gender_accuracy(fold):
    # collect the ids, labels, and predictions for matching
    with open('matched_patients.pickle', 'rb') as f:
        df = pickle.load(f)
    with open(f'ids_pidn_fold_{fold}_{MODEL_ARGS}.pickle', 'rb') as f:
        ids = torch.tensor(pickle.load(f))
    with open(f'labels_fold_{fold}_{MODEL_ARGS}.pickle', 'rb') as f:
        labels = CPU_Unpickler(f).load()
        labels = torch.tensor(np.vstack(labels))
    with open(f'final_preds_fold_{fold}_{MODEL_ARGS}.pickle', 'rb') as f:
        preds = CPU_Unpickler(f).load()
        preds = torch.tensor(np.vstack(preds))
    # flatten from pairs to individual ids
    ids = ids.reshape((-1, 1))
    preds = preds.reshape((-1, 2))
    labels = labels.reshape((-1, 2))
    # prepare df for matching
    df = df.set_index('id')
    df = df.drop(columns={'filename', 'image'})
    # fix only on IDs present in the fold --AND APPLY CORRECT ORDERING!
    df = df.loc[list(ids.numpy().flatten())]
    # store predictions + labels, and boolean flag for prediction correctness
    df['pred'] = list(preds.numpy().round(4))
    df['label'] = list(labels.numpy().astype(int))
    df['correct'] = [(df['pred'].values[i].round() == df['label'].values[i]).all() for i in range(len(df))]
    original_acc = df['correct'].sum() / df['correct'].count()
    # create gender aggregation and compute accuracy over gender
    df = df.groupby('gender').agg({'correct':'mean'})
    df.columns = [f'Fold {fold} - Accuracy']
    df.loc['combined'] = [original_acc]
    df = df.round(4)
    print(df)
    return df

In [5]:
for fold in range(NUM_FOLDS):
    get_gender_accuracy(fold)

          Fold 0 - Accuracy
gender                     
0                    0.3026
1                    0.1071
combined             0.2500
          Fold 1 - Accuracy
gender                     
0                    0.3371
1                    0.0000
combined             0.2885
          Fold 2 - Accuracy
gender                     
0                    0.4176
1                    0.0000
combined             0.3654
          Fold 3 - Accuracy
gender                     
0                    0.3133
1                    0.0952
combined             0.2692
          Fold 4 - Accuracy
gender                     
0                    0.3784
1                    0.1786
combined             0.3235
